# **Using Special Ordered Set (SOS) constraints**

***sos.ipynb***

Construct a problem from scratch with variables of various types. Adds Special Ordered Sets (SOSs) and shows how to retrieve such modeling objects once it has been added to the problem using the API functions.

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://community.fico.com/s/contentdocument/06980000002h0i5AAA) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

Start by importing the xpress Python package, create an Xpress problem, and add all variables used in this example plus a linear constraint over these variables.

In [1]:
import xpress as xp

m = xp.problem()

m.controls.miprelstop = 0

# All variables used in this example
v1 = m.addVariable(lb=0, ub=7, vartype=xp.continuous)
v2 = m.addVariable(lb=1, ub=10, threshold=7, vartype=xp.semicontinuous)
y = [m.addVariable(name="y{0}".format(i)) for i in range(2)]

m.addConstraint(v1 + v2 >= xp.Sum(y[i] for i in range(2)))

The next cell adds an SOS type 1 constraint using the [problem.addSOS()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addSOS.html) method.

The first argument corresponds to the list of variables composing the SOS constraint, and the second argument the list of weights (one per variable) that define the order for SOS constraints (used in branching for both types).

In [2]:
s = m.addSOS([v1, v2], [1, 0], name="mynewsos", type=1)

print("SOS:", s.name)

SOS: mynewsos mynewsos


The following sets an objective, solves the problem, displays some output and exports the model in LP format.


In [3]:
m.setObjective(xp.Sum([y[i] for i in range(2)]), sense=xp.maximize)

m.optimize()

print("v1: ", m.getSolution(v1),
      ", v2: ", m.getSolution(v2),
      "; sol vector: ", m.getSolution(),
      "; obj: ", m.attributes.objval,
      sep="")  # default separator between strings is " "

m.write("restriction", "lp")

FICO Xpress v9.5.1, Community, solve started 16:50:24, Jan 30, 2025
Heap usage: 388KB (peak 388KB, 102KB system)
Maximizing MILP noname using up to 20 threads and up to 31GB memory, with these control settings:
OUTPUTLOG = 1
MIPRELSTOP = 0
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
         1 rows            4 cols            4 elements         2 entities
         1 sets            2 setelem
Presolved problem has:
         0 rows            2 cols            0 elements         2 entities
         1 sets            2 setelem
Presolve finished in 0 seconds
Heap usage: 394KB (peak 402KB, 107KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [      0.0,       0.0]
  RHS and bounds [min,max] : [ 7.00e+00,  1.00e+01] / [ 7.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Will tr

In the following cell, the method [problem.delSOS()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.delSOS.html) is used to delete the previously added SOS1 constraint by passing its object as an argument.

The deletion can be confirmed by comparing the exported LP files created before and after the SOS constraint is deleted.

In [4]:
m.delSOS(s)
m.write("restriction-noSOS", "lp")

m.optimize()

FICO Xpress v9.5.1, Community, solve started 16:50:24, Jan 30, 2025
Heap usage: 2644KB (peak 5484KB, 107KB system)
Maximizing MILP noname using up to 20 threads and up to 31GB memory, with these control settings:
OUTPUTLOG = 1
MIPRELSTOP = 0
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
         1 rows            4 cols            4 elements         1 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 2645KB (peak 5484KB, 107KB system)
Will try to keep branch and bound tree memory usage below 23.2GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  17.000000   .0000000
                        
------- optimal --------
Concurrent statistics:
           Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
   

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)